# Triangle Price Patterns

In [325]:
from datetime import date, datetime, timedelta

In [673]:
import pandas as pd
import yfinance as yf

equity1 = yf.Ticker('BAJAJFINSV.NS')
df = equity1.history(period='50D',interval='1H',actions=False) 
df.reset_index(inplace=True)

In [655]:
df.columns=['datetime', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()

datetime    0
open        0
high        0
low         0
close       0
volume      0
dtype: int64

In [656]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,2,2), axis=1)

## Pivot Candles Vizualisation

In [657]:
import numpy as np
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [658]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df[0:5000]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [674]:
df[df['datetime'] > '2022-08-30']

KeyError: 'datetime'

In [661]:
import numpy as np
#from matplotlib import pyplot
from scipy.stats import linregress
#df = df[(df['datetime'] <(df['datetime'].max() - timedelta(days=1)))]

backcandles =df.shape[0] - (df[df['low']==df['low'].max()].index[0])
candleid = df.shape[0] -1

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])

for i in range(candleid-backcandles, candleid+1):
    if df.iloc[i].pivot == 1:
        minim = np.append(minim, df.iloc[i].low)
        xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
    if df.iloc[i].pivot == 2:
        maxim = np.append(maxim, df.iloc[i].high)
        xxmax = np.append(xxmax, i) # df.iloc[i].name

slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)


In [676]:
slmin

-7.857766523577175

In [675]:
slmax

-16.92829271963574

In [642]:
slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)

In [662]:
df['upper_point'] = (df.index * slmax ) + intercmax
df['lower_point'] = (df.index * slmin ) + intercmin

In [672]:
df.tail()

,datetime,open,high,low,close,volume,pivot,pointpos,upper_point,lower_point
291,2022-08-29 11:15:00+05:30,16000.000000,16087.700195,15991.049805,16050.000000,20516,0,NaN,16134.002178,15981.490386
292,2022-08-29 12:15:00+05:30,16050.000000,16188.400391,16041.000000,16082.799805,39896,2,16188.401391,16117.073885,15973.632619
293,2022-08-29 13:15:00+05:30,16082.799805,16148.000000,16080.000000,16111.000000,21057,0,NaN,16100.145593,15965.774853
294,2022-08-29 14:15:00+05:30,16110.849609,16115.950195,16054.349609,16061.000000,25972,0,NaN,16083.217300,15957.917086
295,2022-08-29 15:15:00+05:30,16063.150391,16175.500000,16054.000000,16149.950195,29439,1,16053.999000,16066.289007,15950.059319


In [671]:
fig = go.Figure(data=[go.Candlestick(x=df.index[-20:],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.add_scatter(x=df.index[-20:], y=df['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot")




fig.add_trace(go.Scatter(x=df.index[-20:], y=df['lower_point'], mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=df.index[-20:], y=df['upper_point'], mode='lines', name='max slope'))
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [276]:
df.to_clipboard()

In [275]:
linregress(df.iloc[1500:1730].index,df['open'].iloc[1500:1730])

LinregressResult(slope=-0.0227259174021372, intercept=301.59229754244717, rvalue=-0.8799868094189903, pvalue=1.156706542027091e-75, stderr=0.0008123996544102351)

In [231]:
backcandles =230

candleid = 1730

RangeIndex(start=1118, stop=1140, step=1)

In [21]:
import numpy as np
from matplotlib import pyplot
from scipy.stats import linregress

backcandles= 20

for candleid in range(20, len(df)):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
    
    if (xxmax.size <3 and xxmin.size <3) or xxmax.size==0 or xxmin.size==0:
        continue
    
    #slmin, intercmin = np.polyfit(xxmin, minim,1) #numpy
    #slmax, intercmax = np.polyfit(xxmax, maxim,1)

    slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
    slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)
        
    #if abs(rmax)>=0.7 and abs(rmin)>=0.7 and abs(slmin)<=0.00001 and slmax<-0.0001:
    #if abs(rmax)>=0.7 and abs(rmin)>=0.7 and slmin>=0.0001 and abs(slmax)<=0.00001:
    if abs(rmax)>=0.9 and abs(rmin)>=0.9 and slmin>=0.0001 and slmax<=-0.0001:
        print(rmin, rmax, candleid)
        break
        
    if candleid % 1000 == 0:
        print(candleid)

/Users/vaigupta/Documents/Notebooks/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:116: RuntimeWarning:

invalid value encountered in double_scalars

/Users/vaigupta/Documents/Notebooks/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:126: RuntimeWarning:

invalid value encountered in sqrt

/Users/vaigupta/Documents/Notebooks/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:128: RuntimeWarning:

invalid value encountered in double_scalars



1.0 -0.9987777546868617 233


In [10]:
df.head(50)

,datetime,open,high,low,close,volume,pivot,pointpos
0,2022-07-21 09:20:00+05:30,164.550003,164.699997,164.000000,164.000000,106149,0,NaN
1,2022-07-21 09:25:00+05:30,164.149994,164.250000,164.000000,164.149994,90620,0,NaN
2,2022-07-21 09:30:00+05:30,164.149994,164.350006,163.699997,164.050003,115592,0,NaN
3,2022-07-21 09:35:00+05:30,164.000000,164.350006,164.000000,164.100006,48980,0,NaN
4,2022-07-21 09:40:00+05:30,164.000000,164.250000,163.750000,163.949997,76067,0,NaN
5,2022-07-21 09:45:00+05:30,164.050003,164.350006,164.000000,164.350006,71015,0,NaN
6,2022-07-21 09:50:00+05:30,164.250000,164.750000,164.050003,164.699997,55850,0,NaN
7,2022-07-21 09:55:00+05:30,164.649994,165.250000,164.649994,165.100006,147958,0,NaN
8,2022-07-21 10:00:00+05:30,165.149994,166.100006,165.149994,165.750000,303505,2,166.101006
9,2022-07-21 10:05:00+05:30,165.750000,165.750000,165.149994,165.600006,66341,0,NaN
